# Premature babies | gestational age & Cooling therapy treatment

In [78]:
headers = {
  'Accept': 'application/fhir+json; fhirVersion=4.0',
  'Content-Type': 'application/fhir+json; fhirVersion=4.0',
  'Cookie': 'ARRAffinity=a6e48b9e9d2653435be7b61998d8624b44115214104213d6c8b8c526cc56dc70; ARRAffinitySameSite=a6e48b9e9d2653435be7b61998d8624b44115214104213d6c8b8c526cc56dc70'
}

In [79]:
import uuid
import pandas as pd
import requests
import json

def generate_observation(datetime, value):
    ID = str(uuid.uuid4())
    observation = {
        "resourceType": "Observation",
        "id": ID,
        "status": "final",
        "category": [
            {
                "coding": [
                    {
                        "system": "gestational-age",
                        "code": "gestational-age",
                        "display": "Gestational age"
                    }
                ]
            }
        ],
        "code": {
            "coding": [
                {
                    "system": "gestAge",
                    "code": "gestAge",
                    "display": "Gestational Age"
                }
            ],
            "text": "Gestational Age"
        },
        "subject": {
            "reference": "https://firelyasserver.azurewebsites.net/Patient/mom"
        },
        "encounter": {
            "reference": "urn:uuid:103c7290-d241-84b6-ee1f-a845af61f414"
        },
        "effectiveDateTime": datetime #"2020-08-31T22:13:22+02:00",
        ,
        "issued": "2020-08-31T22:13:22.922+02:00",
        "valueQuantity": {
            "value": value,
            "unit": "weeks"
        }
    }
    url = f"https://firelyasserver.azurewebsites.net/Observation/{ID}"
    payload = json.dumps(observation)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Not created')
        print(response.status_code)
        print(payload)
    return observation 

In [80]:
def generate_procedure_coolingtherapy(datetime):
    ID = str(uuid.uuid4())
    procedure = {
                "resourceType": "Procedure",
                "id": ID,
                "status": "completed",
                "code": {
                    "coding": [
                        {
                            "system": "CT",
                            "code": "CT",
                            "display": "Cooling therapy"
                        }
                    ],
                    "text": "Cooling therapy"
                },
                "subject": {
                    "reference": "https://firelyasserver.azurewebsites.net/Patient/mom"
                },
                "encounter": {
                    "reference": "no-ref"
                },
                "performedDateTime": datetime,
                "location": {
                    "reference": "AMC",
                    "display": "AMC"
                }
            }
    url = f"https://firelyasserver.azurewebsites.net/Procedure/{ID}"
    payload = json.dumps(procedure)
    response = requests.request("PUT", url, data=payload, headers=headers)
    if response.status_code != 201 and response.status_code != 200:
        print('Not created')
        print(payload)
    return procedure

In [81]:
# result = requests.put('{{BASE_URL}}/Observation/{{id}}', payload =json.dumps({"resourceType": "Observation", "id": "f8890f21-fda3-0744-12c3-bcff3c9d5b75", "meta": {"profile": ["http://hl7.org/fhir/us/core/StructureDefinition/us-core-smokingstatus", "http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-social-history%22]%7D, "status": "final", "category": [{"coding": [{"system": "http://terminology.hl7.org/CodeSystem/observation-category", "code": "social-history", "display": "Social history"}]}], "code": {"coding": [{"system": "http://loinc.org/", "code": "72166-2", "display": "Tobacco smoking status"}], "text": "Tobacco smoking status"}, "subject": {"reference": "urn:uuid:2741c5d6-4b72-1ed4-47c8-600f77941b1e"}, "encounter": {"reference": "urn:uuid:9822b903-8cd5-e853-114f-bf01c9f9ea6d"}, "effectiveDateTime": "2023-04-02T05:02:09+02:00", "issued": "2023-04-02T05:02:09.441+02:00", "valueCodeableConcept": {"coding": [{"system": "http://snomed.info/sct", "code": "266919005", "display": "Never smoked tobacco (finding)"}], "text": "Never smoked tobacco (finding)"}}))

In [82]:
import datetime
import random
import pytz
import pandas as pd

df1 = pd.DataFrame(columns=['year', 'gestational age'])
df2 = pd.DataFrame(columns=['year'])

def generate_random_datetime(year):
    start_date = datetime.datetime(year, 1, 1)
    end_date = datetime.datetime(year, 12, 31)

    # Calculate the total number of seconds within the specified year
    total_seconds = (end_date - start_date).total_seconds()

    # Generate a random duration within the specified year
    random_duration = random.randint(0, int(total_seconds))

    # Add the random duration to the start date
    random_datetime = start_date + datetime.timedelta(seconds=random_duration)

    # Convert the random datetime to the desired format
    formatted_datetime = random_datetime.strftime("%Y-%m-%dT%H:%M:%S")
    formatted_datetime = str(formatted_datetime)+'+02:00'

    return formatted_datetime

def generate_random_gestational_age():
    # Generate a random gestational age with less prematurity during COVID times
    if random.random() < 0.7:  # 70% chance of being full term
        # source: https://pathways.org/adjusting-age-for-prematurity/
        gestational_age = random.randint(38, 42)  # Full-term range (38-42 weeks)
    else:
        gestational_age = random.randint(28, 37)  # Preterm range (28-37 weeks)

    return gestational_age

def generate_dfs(datetime, gestational_age):
    # Placeholder function to generate an observation
    # Replace this function with your actual implementation
    # You can access the datetime and gestational_age variables here
    # to generate the observation data for each patient
    #print(f"Observation generated for datetime: {datetime} and gestational age: {gestational_age}")
    
    df1.loc[len(df1)] = {'year': str(datetime)[:4], 'gestational age': gestational_age}

for year in range(2017, 2024):
    for _ in range(500):
        random_datetime = generate_random_datetime(year)
        random_gestational_age = generate_random_gestational_age()

        if year == 2020 or year == 2021 or year == 2022:  # Considering the COVID period
            random_gestational_age += random.randint(2, 4)  # Increase the gestational age by 2 to 4 weeks
            
            if random.random() < 0.80:  # 80% chance of cooling therapy during covid
                generate_procedure_coolingtherapy(random_datetime)
                df2.loc[len(df2)] = {'year': str(random_datetime)[:4]}

        
        generate_dfs(random_datetime, random_gestational_age)
        generate_observation(random_datetime, random_gestational_age)
        
        if year == 2017 or year == 2018 or year == 2019 or year == 2023:
            if random.random() < 0.60:  # 60% chance of cooling therapy during covid
                generate_procedure_coolingtherapy(random_datetime)
                df2.loc[len(df2)] = {'year': str(random_datetime)[:4]}
        
